<a href="https://colab.research.google.com/github/ommore86/Machine-Learning/blob/main/ML_5_K_Fold_Cross_Validation_with_bias_variance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving netflix_customer_churn.csv to netflix_customer_churn.csv


In [ ]:
df = pd.read_csv('netflix_customer_churn.csv')
df.head()

,customer_id,age,gender,subscription_type,watch_hours,last_login_days,region,device,monthly_fee,churned,payment_method,number_of_profiles,avg_watch_time_per_day,favorite_genre
0,a9b75100-82a8-427a-a208-72f24052884a,51,Other,Basic,14.73,29,Africa,TV,8.99,1,Gift Card,1,0.49,Action
1,49a5dfd9-7e69-4022-a6ad-0a1b9767fb5b,47,Other,Standard,0.70,19,Europe,Mobile,13.99,1,Gift Card,5,0.03,Sci-Fi
2,4d71f6ce-fca9-4ff7-8afa-197ac24de14b,27,Female,Standard,16.32,10,Asia,TV,13.99,0,Crypto,2,1.48,Drama
3,d3c72c38-631b-4f9e-8a0e-de103cad1a7d,53,Other,Premium,4.51,12,Oceania,TV,17.99,1,Crypto,2,0.35,Horror
4,4e265c34-103a-4dbb-9553-76c9aa47e946,56,Other,Standard,1.89,13,Africa,Mobile,13.99,1,Crypto,2,0.13,Action


In [ ]:
df = df.dropna()
target = 'churned'

# Define features (excluding target)
features = df.drop(columns=[target]).columns

# Select features for X
X = df[features]
y = df[target]

# Convert categorical variables to dummy/one-hot encoding manually
X = pd.get_dummies(X, drop_first=True)

In [ ]:
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor

# Assuming X, y are defined and preprocessed

kf = KFold(n_splits=10, shuffle=True, random_state=42)
rf_model = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)

fold_scores = cross_val_score(rf_model, X, y, scoring='neg_mean_squared_error', cv=kf, n_jobs=-1)
positive_mse = -fold_scores
print(f"Fold scores (MSE): {positive_mse}")

bias_per_fold = []
variance_per_fold = []

for train_idx, test_idx in kf.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)

    # Bias for this fold: mean(predictions) - mean(true values)
    bias = np.mean(y_pred) - np.mean(y_test)
    bias_per_fold.append(bias)

    # Variance for this fold: variance of predictions for the fold
    variance = np.var(y_pred)
    variance_per_fold.append(variance)

print(f"Bias per fold: {bias_per_fold}")
print(f"Variance per fold: {variance_per_fold}")

Fold scores (MSE): [0.019132  0.009868  0.0059952 0.0054216 0.0108968 0.010824  0.010592
 0.0187024 0.0213952 0.0127648]
Bias per fold: [np.float64(-0.0037999999999999146), np.float64(-0.002120000000000122), np.float64(0.0020799999999999708), np.float64(-0.0009199999999999764), np.float64(0.0015600000000000058), np.float64(-0.008399999999999963), np.float64(-0.0048000000000000265), np.float64(-0.009919999999999929), np.float64(-0.0023200000000000998), np.float64(-0.0030399999999999316)]
Variance per fold: [np.float64(0.23540556), np.float64(0.23910926560000004), np.float64(0.2367324736), np.float64(0.2352284336), np.float64(0.23723708640000007), np.float64(0.23934303999999998), np.float64(0.23588976), np.float64(0.2376286336), np.float64(0.23290085759999993), np.float64(0.23542099840000003)]
